In [363]:
# Кодирование – выбор «генетического кода»
# Особь – битовая последовательность размера n (кол-во грузов)
# 1. Начальная популяция – кол-во особей всегда = 200:
# 1.2 жадный выбор, начиная со случайного груза
# 2. Отбор особей для скрещивания :
# 2.2 выбрать только 20% самых приспособленных особей
# 3. Скрещивание (кроссинговер) между выбранными особями. Каждая особь скрещивается 1 раз за 1 поколение, 1 пара дает 2 потомка:
# 3.2 однородный (каждый бит от случайно выбранного родителя)
# 4. Мутация:
# 4.2 случайное изменение 3х битов у 5% особей
# 5. Формирование новой популяции (кол-во особей - константа)
# 5.2 «штраф» за «старость» -20% функции приспособленности, выбор лучших
# 6. Оценка результата
# Наступила сходимость (функция приспособленности лучшей особи в популяциях отличается не более, чем на 10%)
# или прошло 500 поколений

In [364]:
import random
import math

In [365]:
with open("11.txt") as f:
    line1 = f.readline().rstrip() # сразу удаляем перенос строки в конце
    MAX_WEIGHT = int(line1.split(' ')[0]) # грузоподъемность
    MAX_VOLUME = int(line1.split(' ')[1]) # вместимость
    items = {}
    for num, line in enumerate(f):
        cur = list(map(float, line.rstrip().split(' ')))
        # Назначаем каждому айтему условную ценность
        cur.append(round(cur[2] / ( (cur[0]/MAX_WEIGHT) + (cur[1]/MAX_VOLUME) )))
        items[num] = tuple(cur)

In [426]:
def evaluate(individual):
    weight = 0
    volume = 0
    value = 0
    for num, item_descr in enumerate(individual):
        if item_descr == 1:
            weight += items[num][0]
            volume += items[num][1]
            value += items[num][2]
    if weight > MAX_WEIGHT or volume > MAX_VOLUME:
        weight = 1000000
        volume = 1000000
        value = 0
    return weight, volume, value

def addItem(individual, item_num):
    individual[item_num] = True
    return

def removeItem(individual, item_num):
    individual[item_num] = False
    return

def fitness(individual):
    weight, volume, value = evaluateKnapsack(individual)
    if (weight == 0) or (volume == 0):
        return 0
    else:
        ftn = value + value /((weight/MAX_WEIGHT) + (volume/MAX_VOLUME))
    return ftn

def printInfoInd(individual):
    weight, volume, value = evaluateKnapsack(individual)
    weight_percent = str(round(100*weight/MAX_WEIGHT, 1))+'%'
    volume_percent = str(round(100*volume/MAX_VOLUME, 1))+'%'
    print(round(value, 1), '(', weight_percent, volume_percent, ')', fitness(individual))
    return

def printInfoPop(population):
    for individual in population:
        printInfoInd(individual)

In [439]:
# Создаем популяцию
POPULATION_SIZE = 20
population = [[False,] * len(items) for i in range(POPULATION_SIZE)]

# Индексируем айтемы для ускорения жадного алгоритма
items_indexed = list(items.items())
items_indexed.sort(key=lambda x: x[1][3], reverse=True)

# Жадным алгоритмом набираем айтемы
for ind in population:
    rand = random.randint(0, len(items) - 1)
    ind[rand] = 1
    for item in items_indexed:
        addItem(ind, item[0])
        if evaluate(ind)[2] == 0:
            removeItem(ind, item[0])

In [440]:
generations_number = 30

print("before:")
population.sort(key=lambda x: fitness(x), reverse=True)
printInfoPop(population)

for i in range(generations_number):
    # Выбрать только 20% самых приспособленных особей
    population.sort(key=lambda x: fitness(x), reverse=True)
    attractive = population[:math.floor(len(population)/5)]
    # Каждая особь скрещивается не более 1 раза
    random.shuffle(attractive)
    att_len = math.floor(len(attractive)/2)
    sex1, sex2 = attractive[:att_len], attractive[att_len:]
    children = []
    for parent1, parent2 in zip(sex1, sex2):
        # Однородный отбор (каждый бит от случайно выбранного родителя)
        child = []
        for bit1, bit2 in zip(parent1, parent2):
            bitval = random.choice([bit1, bit2])
            child.append(bitval)
        children.append(child)
        
    # Мутации
    not_mutated = list(range(len(population)))
    for ind in population:
        if random.random() < 0.05:
            rand_bits = random.sample(range(0, len(ind)), 3)
            for bitnum in rand_bits:
                ind[bitnum] = not ind[bitnum]
                
    # Формирование новой популяции
    candidates = []
    candidates += map(lambda ind: tuple([ind, fitness(ind)]), children)
    candidates += map(lambda ind: tuple([ind, fitness(ind) * 0.8]), population)
    candidates.sort(key=lambda x: x[1], reverse=True)
    alive = candidates[:POPULATION_SIZE]
    population = list(map(lambda x: x[0], alive))
    
print("after:")
printInfoPop(population)

before:
4781.0 ( 99.7% 100.0% ) 7175.183359013867
4781.0 ( 99.7% 100.0% ) 7175.183359013867
4781.0 ( 99.7% 100.0% ) 7175.183359013867
4781.0 ( 99.7% 100.0% ) 7175.183359013867
4781.0 ( 99.7% 100.0% ) 7175.183359013867
4656.0 ( 96.2% 95.0% ) 7090.8525684862625
4656.0 ( 96.2% 95.0% ) 7090.8525684862625
4656.0 ( 96.2% 95.0% ) 7090.8525684862625
4656.0 ( 96.2% 95.0% ) 7090.8525684862625
4656.0 ( 96.2% 95.0% ) 7090.8525684862625
4656.0 ( 96.2% 95.0% ) 7090.8525684862625
4656.0 ( 96.2% 95.0% ) 7090.8525684862625
4656.0 ( 96.2% 95.0% ) 7090.8525684862625
4656.0 ( 96.2% 95.0% ) 7090.8525684862625
4659.0 ( 97.1% 95.0% ) 7084.8821644570835
4642.0 ( 95.4% 95.8% ) 7068.912971204891
4642.0 ( 95.4% 95.8% ) 7068.912971204891
4652.0 ( 98.6% 100.0% ) 6994.850501685198
4618.0 ( 99.6% 96.7% ) 6971.013417645445
4518.0 ( 100.0% 91.7% ) 6875.311999143778
after:
4781.0 ( 99.7% 100.0% ) 7175.183359013867
4781.0 ( 99.7% 100.0% ) 7175.183359013867
4781.0 ( 99.7% 100.0% ) 7175.183359013867
4781.0 ( 99.7% 100.0% 